# Defining rho for the Black process

(Based on a question by DPaulino on the QuantLib mailing list. Thanks!)

In [1]:
from QuantLib import *

In [2]:
today = Date(24,12,2016)
Settings.instance().evaluationDate = today

#### The dangers of generalization

QuantLib provides a few classes to represent specific cases of the Black-Scholes-Merton process; for instance, the `BlackScholesProcess` class assumes that there are no dividends, and the `BlackProcess` class that the cost of carry is equal to 0.  It is the latter, or rather a glitch in it, that is the subject of this notebook.

All such classes inherit from a base `GeneralizedBlackScholesProcess` class (I know, we're not that good at naming things) that models the more general case in which the underlying stock has a continuous dividend yield.  The specific cases are implemented by inheriting from this class and setting a constraint on the dividends $q(t)$: for the Black-Scholes process, $q(t) = 0$; and for the Black process, $q(t) = r(t)$, which makes the cost of carry $b$ equal 0.

We can check the constraint by creating two instances of such processes.  Here are the quotes and term structures we'll use to model the dynamics of the underlying:

In [3]:
u = SimpleQuote(100.0)
r = SimpleQuote(0.01)
sigma = SimpleQuote(0.20)

risk_free_curve = FlatForward(today, QuoteHandle(r), Actual360())
volatility = BlackConstantVol(today, TARGET(), QuoteHandle(sigma), Actual360())

The constructor of the `BlackScholesProcess` class doesn't take a dividend yield, and sets it to 0 internally:

In [4]:
process_1 = BlackScholesProcess(QuoteHandle(u),
                                YieldTermStructureHandle(risk_free_curve),
                                BlackVolTermStructureHandle(volatility))

print(process_1.dividendYield().zeroRate(1.0, Continuous))

0.000000 % Actual/365 (Fixed) continuous compounding


The constructor of the `BlackProcess` class doesn't take a dividend yield either, and sets its handle as a copy of the risk free handle: 

In [5]:
process_2 = BlackProcess(QuoteHandle(u),
                         YieldTermStructureHandle(risk_free_curve),
                         BlackVolTermStructureHandle(volatility))

print(process_2.riskFreeRate().zeroRate(1.0, Continuous))
print(process_2.dividendYield().zeroRate(1.0, Continuous))

1.000000 % Actual/360 continuous compounding
1.000000 % Actual/360 continuous compounding


Now, the above processes can be used to price options on underlyings behaving accordingly; the first process describes, e.g., a stock that doesn't pay any dividends, and the second describes, e.g., a futures.  The classes to use are the same: `EuropeanOption` for the instrument and `AnalyticEuropeanEngine` for the pricing engine.  The constructor of the engine takes an instance of `GeneralizedBlackScholesProcess`, to which both our processes can be converted implicitly.

In [6]:
option_1 = EuropeanOption(PlainVanillaPayoff(Option.Call, 100.0),
                          EuropeanExercise(today+100))
option_1.setPricingEngine(AnalyticEuropeanEngine(process_1))

print(option_1.NPV())

4.337597216336533


In [7]:
option_2 = EuropeanOption(PlainVanillaPayoff(Option.Call, 100.0),
                          EuropeanExercise(today+100))
option_2.setPricingEngine(AnalyticEuropeanEngine(process_2))

print(option_2.NPV())

4.191615257389808


So far, so good.  However, we can see the glitch when we ask the options for their Greeks.  With this particular engine, they're able to calculate them by using closed formulas (none other, of course, that those expressing the derivatives of the Black-Scholes-Merton formula).  As I described in a previous notebook, we can also calculate the Greeks numerically, by bumping the inputs and repricing the option.  If we compare the two approaches, they should yield approximately the same results.

For convenience, I'll define a utility function to calculate numerical Greeks. It takes the option, the quote to change and the amplitude of the bump.

In [8]:
def greek(option, quote, dx):
    x0 = quote.value()
    quote.setValue(x0+dx)
    P_u = option.NPV()
    quote.setValue(x0-dx)
    P_d = option.NPV()
    quote.setValue(x0)
    return (P_u-P_d)/(2*dx)

By passing different quotes, we can calculate different Greeks. Bumping the underlying value will give us the delta, which we can compare to the analytic result:

In [9]:
print(option_1.delta())
print(greek(option_1, u, 0.01))

0.5315063340142601
0.531506323010289


In [10]:
print(option_2.delta())
print(greek(option_2, u, 0.01))

0.5195711146255227
0.5195711052036867


Bumping the volatility gives us the vega...

In [11]:
print(option_1.vega())
print(greek(option_1, sigma, 0.001))

20.96050033373808
20.960499909565833


In [12]:
print(option_2.vega())
print(greek(option_2, sigma, 0.001))

20.938677847075486
20.938677605407463


...and bumping the risk-free rate will give us the rho.

In [13]:
print(option_1.rho())
print(greek(option_1, r, 0.001))

13.559176718080407
13.55917453385036


In [14]:
print(option_2.rho())
print(greek(option_2, r, 0.001))

13.268193390322908
-1.1643375864700545


Whoops.  Not what you might have expected.

#### What's happening here?

The problem is that the engine works with a generic process, and $\rho$ is calculated as

$$
\rho = \frac{\partial}{\partial r} C(u, r, q, \sigma)
$$

where $C$ is the Black-Scholes-Merton formula for the value of the call option.

However, not knowing about the specific process type we passed, the engine doesn't know about the constraint we set on the underlying variables: in this case, that $q = q(r) = r$.  Therefore, the correct value for $\rho$ should be

$$
\rho = \frac{d}{dr} C(u, r, q(r), \sigma) 
    = \frac{\partial C}{\partial r} + \frac{\partial C}{\partial q} \cdot \frac{\partial q}{\partial r}
    = \frac{\partial C}{\partial r} + \frac{\partial C}{\partial q}.
$$

which is the sum of the rho as defined in the engine and the dividend rho.  We can verify this by comparing the above with the numerical Greek:

In [15]:
print(option_2.rho() + option_2.dividendRho())
print(greek(option_2, r, 0.001))

-1.1643375714971693
-1.1643375864700545


Now: is this a bug in the engine?

Well, it might be argued. The engine might detect the case of a Black process and change the calculation of rho accordingly; it's kind of a hack, and there goes the genericity, but it's possible to implement.  However, the above might also happen with a usually well-behaved process if we use the same term structure for $r$ and $q$:

In [16]:
process_3 = BlackScholesMertonProcess(QuoteHandle(u),
                                      YieldTermStructureHandle(risk_free_curve),
                                      YieldTermStructureHandle(risk_free_curve),
                                      BlackVolTermStructureHandle(volatility))
option_3 = EuropeanOption(PlainVanillaPayoff(Option.Call, 100.0),
                          EuropeanExercise(today+100))
option_3.setPricingEngine(AnalyticEuropeanEngine(process_3))

In [17]:
print(option_3.delta())
print(greek(option_3, u, 0.01))

0.5195711146255227
0.5195711052036867


In [18]:
print(option_3.rho())
print(greek(option_3, r, 0.001))
print(option_3.rho() + option_3.dividendRho())

13.268193390322908
-1.1643375864700545
-1.1643375714971693


The issue is not even limited to processes.  You're defining a discount curve as the risk-free rate plus a credit spread? Bumping the risk-free rate will modify both, and your sensitivities will be affected accordingly (even though in this case the effect is probably what you wanted).  In any case, this is something you should be aware of.